# SVM - Climate Sentiment Multiclass Classification
## CS522 Project
SVM with Count Vectorizer

### Dataset: 
https://www.kaggle.com/code/luiskalckstein/climate-sentiment-multiclass-classification

### Imports

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import LinearSVC,SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from Common.DataCenter import data_center
from Common.LSI import SKLearnLSA
from Common.UtilFuncs import DataSize
from Common.UtilFuncs import print_evaluation, EvaluationToDF
import pandas as pd

TrainSizeBaseLine = DataSize.GetTrainSizeBaseline()
TrainSizeWithNoisyData = DataSize.GetTrainSizeWithNoisyData()
TestDataSize = DataSize.GetTestDataSize()
NoiseDataSize = DataSize.GetNoiseDataSize()
ValidationDataSize = DataSize.GetValidationDataSize()

%matplotlib inline
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 134 µs (started: 2022-04-09 16:51:37 +08:00)


### Text preprocessing

In [2]:
# parameter: original X of training set and test set
# return:  vectorised X of training set and test set
def text_preprocessing(X_train, X_test):
    # Convert texts to vectors
    vectorizer = CountVectorizer(min_df = 1, stop_words = 'english')
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    X_train_vec = X_train_vec.astype(float)
    X_test_vec = X_test_vec.astype(float)


    return X_train_vec, X_test_vec


time: 351 µs (started: 2022-04-09 16:51:37 +08:00)


### One-hot encoding, convert the labels to vectors (4 x 1) each

In [3]:
# parameter: original y of training set, original y of test set
# return:  encoded y of training set and test set
def one_hot_encoding(y_train, y_test):
    mlb          = MultiLabelBinarizer()
    y_train_vec  = mlb.fit_transform(map(str, y_train))
    y_test_vec   = mlb.transform(map(str, y_test))
    return y_train_vec, y_test_vec


time: 295 µs (started: 2022-04-09 16:51:37 +08:00)


### Run SVM and evaluate the results

In [4]:
# parameter:  vectorised X and encoded y of training set and test set
def evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec):
    # Run SVM - fit and predict
    SVM             = OneVsRestClassifier(LinearSVC(dual=False, class_weight='balanced'), n_jobs=-1)
    #SVM = OneVsRestClassifier(SVC(gamma='auto'), n_jobs=-1)
    SVM.fit(X_train_vec, y_train_vec)
    prediction      = SVM.predict(X_test_vec)
    print_evaluation(y_test_vec, prediction)
    
    # Evaluate the results
    macro_f1        = f1_score(y_test_vec, prediction, average='macro')
    weighted_f1     = f1_score(y_test_vec, prediction, average='weighted')
    macro_precision = precision_score(y_test_vec, prediction, average='macro')
    macro_recall    = recall_score(y_test_vec, prediction, average='macro')

    return macro_f1, weighted_f1, macro_precision, macro_recall


time: 489 µs (started: 2022-04-09 16:51:37 +08:00)


### Do an experiment

In [5]:
# Parameter: original X,y of training set and test set
def do_experiment(X_train, y_train, X_test, y_test):
    # Convert texts to vectors
    X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
    y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)

    # Run SVM and evaluate the results
    macro_f1, weighted_f1, macro_precision, macro_recall = \
        evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec)

    # Show the indicators
    #print(" macro_f1: %.4f , weighted_f1: %.4f, macro_precision: %.4f, macro_recall: %.4f" %
    #      (macro_f1, weighted_f1, macro_precision, macro_recall))
    return X_train_vec


time: 410 µs (started: 2022-04-09 16:51:37 +08:00)


### Main entry

In [6]:

noisy_set_sizes = {
    'mislabeled' : 5000,   # max size: 15000
    'irrelevant' : 5000,   # max size: 34259
    'translated' : 5000,   # max size: 5000
}

# Load the database and split it into training set, test set, noisy set, validation set
dc = data_center("twitter_sentiment_data_clean.csv", test_size = 4000, validation_size = 1000,
                 noisy_size = noisy_set_sizes['mislabeled'])

print("####################################################")
print("Total data size: ",       dc.get_len())
print("Total train data size: ", dc.get_train_len())
print("Total test data size: ",  dc.get_test_len())

####################################################
Total data size:  40908
Total train data size:  30908
Total test data size:  4000
time: 122 ms (started: 2022-04-09 16:51:37 +08:00)


**Get the test set for evaluation**

In [7]:
X_test, y_test = dc.get_test()


time: 4.3 ms (started: 2022-04-09 16:51:37 +08:00)


**Set distributions of training set.**

In [8]:
# distribution of training set
train_distribution = None


time: 218 µs (started: 2022-04-09 16:51:37 +08:00)


**Prepare the noisy set.**

In [9]:
lstNoisyInfo = [("mislabeled",dc.get_noisy_len())]
print("Noisy set size is %d"                % dc.get_noisy_len())

# add the external noisy data (irrelevant texts)
# distribution of irrelevant noisy
irrelevant_noisy_distribution = [0.25, 0.25, 0.25, 0.25]    # None, if use the distribution of original set
added_size = dc.add_noisy(noisy_source="irrelevant", distribution = irrelevant_noisy_distribution,
                          size = noisy_set_sizes['irrelevant'])
print("%d noisy samples added" % added_size)
lstNoisyInfo.append(("irrelevant",added_size))

# add the external noisy data (translated texts). use the labels of each noisy data
added_size = dc.add_noisy(noisy_source="translated", distribution = "reserve_labels", 
                          size = noisy_set_sizes['translated'])
print("%d noisy samples added" % added_size)
lstNoisyInfo.append(("translated",added_size))

print("Noisy set new size is %d"                % dc.get_noisy_len())



Noisy set size is 5000
5000 noisy samples added
5000 noisy samples added
Noisy set new size is 15000
time: 322 ms (started: 2022-04-09 16:51:37 +08:00)


**Load the database and split it into training set, test set, noisy set, validation set**

**Get the test set for evaluation**

**Run experiments with different training sets, and use the same test set.**

In [10]:
print("-----------------------------------------------")
for size in TrainSizeBaseLine:
    # Get a training set without noisy data
    X_train, y_train = dc.get_train(size, train_distribution)
    print("Training set size: %d samples (%.1f%%): " % (len(X_train), len(y_train)/dc.get_train_len()*100))

    # Do an experiment
    do_experiment(X_train, y_train, X_test, y_test)

print("-----------------------------------------------")
xtrainvec = None
for size in TrainSizeWithNoisyData:
    # Get a noisy training set
    X_train, y_train = dc.get_train_with_noisy(size[0], size[1], train_distribution)
    print("Noisy training set size: %d samples (%d original, %d noisy)" % (len(y_train), size[0], size[1]))

    # Do an experiment
    xtrainvec = do_experiment(X_train, y_train, X_test, y_test)

-----------------------------------------------
Training set size: 2000 samples (6.5%): 
  f1 of classes: [0.254, 0.402, 0.657, 0.599]
  micro_f1: 0.571 , macro_f1: 0.478 , weighted_f1: 0.560, macro_precision: 0.594, macro_recall: 0.421
Training set size: 4000 samples (12.9%): 
  f1 of classes: [0.366, 0.43, 0.701, 0.651]
  micro_f1: 0.618 , macro_f1: 0.537 , weighted_f1: 0.609, macro_precision: 0.613, macro_recall: 0.489
Training set size: 5000 samples (16.2%): 
  f1 of classes: [0.411, 0.448, 0.7, 0.665]
  micro_f1: 0.625 , macro_f1: 0.556 , weighted_f1: 0.619, macro_precision: 0.617, macro_recall: 0.514
Training set size: 8000 samples (25.9%): 
  f1 of classes: [0.424, 0.461, 0.714, 0.676]
  micro_f1: 0.637 , macro_f1: 0.569 , weighted_f1: 0.632, macro_precision: 0.624, macro_recall: 0.531
Training set size: 10000 samples (32.4%): 
  f1 of classes: [0.467, 0.476, 0.715, 0.683]
  micro_f1: 0.644 , macro_f1: 0.585 , weighted_f1: 0.641, macro_precision: 0.633, macro_recall: 0.551
Train

/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


  f1 of classes: [0.358, 0.389, 0.681, 0.619]
  micro_f1: 0.590 , macro_f1: 0.512 , weighted_f1: 0.584, macro_precision: 0.579, macro_recall: 0.463
Noisy training set size: 10000 samples (8000 original, 2000 noisy)


/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


  f1 of classes: [0.436, 0.442, 0.689, 0.635]
  micro_f1: 0.610 , macro_f1: 0.551 , weighted_f1: 0.608, macro_precision: 0.592, macro_recall: 0.515
Noisy training set size: 20000 samples (15000 original, 5000 noisy)


/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


  f1 of classes: [0.417, 0.443, 0.691, 0.644]
  micro_f1: 0.610 , macro_f1: 0.549 , weighted_f1: 0.610, macro_precision: 0.575, macro_recall: 0.526
time: 23 s (started: 2022-04-09 16:51:38 +08:00)


/Users/guopei/miniforge3/envs/tf2/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
